In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import tensorflow as tf
import os
import numpy as np
import tensorflow_io as tfio

from helper_functions import (
    create_spectrogram_features,
    lite_model_from_file_predicts_dataset,
    get_file_size, 
    convert_bytes,
    evaluate_prediction,
    full_int_model_predict
)


from sklearn.metrics import confusion_matrix

In [3]:
desired_length_of_audio = 48000
sample_rate = 16000

In [4]:
# Take all audio from testing dataset and create spectrograms from them
# We will use spectrograms for models testing
directory = 'dataset/testing'

x_data = []
y_data = []
for root, dirs, files in os.walk(directory):
    for file in files:
        full_file_name = os.path.join(root, file)

        if "non_target" in str(full_file_name):
            class_encoded = 0
        elif "target" in str(full_file_name):
            class_encoded = 1

        audio, sr = tf.audio.decode_wav(tf.io.read_file(full_file_name))
        audio = tf.squeeze(audio, axis=-1)
        resampled_audio = tfio.audio.resample(audio, rate_in=48000, rate_out=sample_rate)
        # Prepare log mel spectrogram from audio
        spectrogram_feature = create_spectrogram_features(resampled_audio, desired_length=48000, sample_rate = 16000)
        x_data.append(spectrogram_feature)
        y_data.append(class_encoded)

# input data should be in numpy array, not in list
x_data_np = np.array(x_data)
y_data_np = np.array(y_data)

2024-06-20 10:40:54.580431: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 10:40:54.580822: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-20 10:40:54.732659: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-20 10:40:54.759215: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [5]:
for i in x_data_np:
    print(i.shape)
    break

(184, 80, 1)


### CNN

##### CNN initial model in keras format

In [6]:
cnn_initial_model_path = 'spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_baseline.keras'
convert_bytes(get_file_size(cnn_initial_model_path), "KB")

File size: 335.824 Kilobytes


In [7]:
cnn_initial_model = tf.keras.models.load_model(cnn_initial_model_path)
# input data should be in numpy array
y_pred_prob = cnn_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model (without any additional quantization techniques)

In [19]:
cnn_tflite_model_path = 'spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz.tflite'
convert_bytes(get_file_size(cnn_tflite_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_tflite_model_path, x_data_np, y_data_np)

File size: 103.684 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Post Training Dynamic range quantization

In [20]:
cnn_drq_model_path = 'spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_drq.tflite'
convert_bytes(get_file_size(cnn_drq_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_drq_model_path, x_data_np, y_data_np)

File size: 29.508 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Float 16 quantization

In [29]:
cnn_float16q_model_path = 'spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16KHz_float16q.tflite'
convert_bytes(get_file_size(cnn_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_float16q_model_path, x_data_np, y_data_np)

File size: 54.879 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Full int quantization

In [13]:
cnn_full_int_q_model_path = 'spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(cnn_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(cnn_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 29.898 Kilobytes
Accuracy: 98.78%
Recall: 98.47%
Precision: 97.83%
F1-score: 98.15%


### SqueezeNet

##### SqueezeNet initial model in keras format

In [30]:
squeezenet_initial_model_path = 'spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras'
convert_bytes(get_file_size(squeezenet_initial_model_path), "KB")

File size: 8687.448 Kilobytes


In [31]:
squeezenet_initial_model = tf.keras.models.load_model(squeezenet_initial_model_path)
# input data should be in numpy array
y_pred_prob = squeezenet_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model (without any additional quantization techniques)

In [32]:
squeezenet_tflite_model_path = 'spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz.tflite'
convert_bytes(get_file_size(squeezenet_tflite_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_model_path, x_data_np, y_data_np)

File size: 2856.07 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model + Post Training Dynamic range quantization

In [33]:
squeezenet_tflite_drq_model_path = 'spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_drq.tflite'
convert_bytes(get_file_size(squeezenet_tflite_drq_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_drq_model_path, x_data_np, y_data_np)

File size: 770.258 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model + Float 16 quantization

In [34]:
squeezenet_float16q_model_path = 'spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_float16q.tflite'
convert_bytes(get_file_size(squeezenet_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(squeezenet_float16q_model_path, x_data_np, y_data_np)

File size: 1442.977 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### CNN tf Lite model + Full integer quantization

In [14]:
squeezenet_full_int_q_model_path = 'spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(squeezenet_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(squeezenet_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 807.539 Kilobytes
Accuracy: 98.21%
Recall: 97.82%
Precision: 96.76%
F1-score: 97.29%
